In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pip-packages-icr/pip-packages/tabpfn-0.1.9-py3-none-any.whl
/kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt
/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
!pip install tabpfn --no-index --find-links=file:///kaggle/input/pip-packages-icr/pip-packages
!mkdir -p /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff
!cp /kaggle/input/pip-packages-icr/pip-packages/prior_diff_real_checkpoint_n_0_epoch_100.cpkt /opt/conda/lib/python3.10/site-packages/tabpfn/models_diff/

Looking in links: file:///kaggle/input/pip-packages-icr/pip-packages
Processing /kaggle/input/pip-packages-icr/pip-packages/tabpfn-0.1.9-py3-none-any.whl


In [3]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import KFold , StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# ML
from sklearn.ensemble import RandomForestClassifier  # Bagging
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier            # GBM
from sklearn.impute import KNNImputer
import lightgbm as lgb
import catboost as cb
from tabpfn import TabPFNClassifier
import tqdm



from keras.optimizers import Adam , SGD , RMSprop
from keras.callbacks import ReduceLROnPlateau
from xgboost import XGBClassifier

from functools import partial
from imblearn.over_sampling import SMOTE

# AutoML framework
import optuna
from optuna.samplers import TPESampler


import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import KFold , StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# ML
from sklearn.ensemble import RandomForestClassifier  # Bagging
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier            # GBM
from sklearn.impute import KNNImputer
import lightgbm as lgb
import catboost as cb
from tabpfn import TabPFNClassifier
import tqdm



from keras.optimizers import Adam , SGD , RMSprop
from keras.callbacks import ReduceLROnPlateau
from xgboost import XGBClassifier

from functools import partial
from imblearn.over_sampling import SMOTE

# AutoML framework
import optuna
from optuna.samplers import TPESampler


import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
# set configs
is_tuning = True
is_scaling = True
is_pca = False
apply_vif = True
feature_selection = True
is_cuml = True
is_debug = True
sampling_method = 'hybrid' # 'under' or 'over'

# Keras model compile
learning_rate = 1e-2
batch_size = 32
epochs = 10


#MY CONFIG
balance_dataset = False # Upper dataset
apply_scaler = False


if is_tuning:
    sampler = TPESampler(seed=42)  # Make the sampler behave in a deterministic way.
    
if feature_selection:
    m = 20
    
if is_tuning:
    n_trials=50
    

In [6]:
def balance_logloss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    y_pred /= np.sum(y_pred, axis=1)[:, None]
    nc = np.bincount(y_true)
    
    logloss = (-1/nc[0]*(np.sum(np.where(y_true==0,1,0) * np.log(y_pred[:,0]))) - 1/nc[1]*(np.sum(np.where(y_true!=0,1,0) * np.log(y_pred[:,1])))) / 2
    
    return logloss

In [7]:
#bring in the six packs
train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')


print("Full train dataset shape is {}".format(train.shape))

Full train dataset shape is (617, 58)


In [8]:
train = train.drop(columns=["Id"])
test = test.drop(columns=["Id"])

In [9]:
train = train.drop(columns=["CL" , "DV" , "BC" , "EH" , "FD ", "CS"])
test = test.drop(columns=["CL" , "DV" , "BC" , "EH" , "FD ", "CS"])

In [10]:
lb = LabelEncoder()

train.EJ = lb.fit_transform(train.EJ)  # A->0, B->1
test.EJ = lb.fit_transform(test.EJ)  # A->0, B->1

In [11]:
#missing data
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
EL,60,0.097245
BQ,60,0.097245
CC,3,0.004862
FS,2,0.003241
CB,2,0.003241
FL,1,0.001621
FC,1,0.001621
DU,1,0.001621
GL,1,0.001621
EG,0,0.000000


In [12]:
imp = KNNImputer()
labels = train["Class"]
train= train.drop(columns="Class")
data = imp.fit_transform(train)
tmp = pd.DataFrame(columns=train.columns, data=data)
train = pd.concat([tmp, labels], axis=1)

train.isnull().sum().max() #just checking that there's no missing data missing...

0

In [13]:
#GET THE Scaler ( NOT APPLY YET)
scaler = StandardScaler()
scaler.fit_transform(train.drop(columns="Class"))

if apply_scaler:
    test = scaler.transform(test)

In [14]:
y = train["Class"]
X= train.drop(columns="Class")

In [15]:
# UPPER THE DATASET
if balance_dataset:
    
    # Step 1: Identify minority and majority classes
    minority_class = X[y == 1]

    # Step 2: Compute the difference in the number of samples
    num_minority = len(minority_class)
    num_majority = len(X) - num_minority
    diff_samples = num_majority - num_minority

    print(f'num_minority {num_minority}')
    print(f'num_majority {num_majority}')
    print(f'diff_samples {diff_samples}')


    # Step 3-4: Apply SMOTE to generate synthetic samples
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_smote, y_smote = smote.fit_resample(X, y)

    #Attention X_smote will be the the whole dataset include the y value

    # Step 5: Create a new DataFrame with the upsampled dataset
    upsampled_df = pd.DataFrame(X_smote, columns=X.columns)
    upsampled_df["Class"] = y_smote

    # Update the dataset with new upper sampling
    train = upsampled_df
    y = train["Class"]
    X= train.drop(columns="Class")

In [16]:
class KFoldModel:
    def __init__(self, n_splits, model_type, params):
        self.n_splits = n_splits
        self.model_type = model_type
        self.params = params
        self.models = []
        self.fitted = False

    def fit(self, X, y):
        self.models = []
        oof = np.zeros((len(train), 2))
        skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True,random_state=42)
        
        print(f'################{self.model_type} train starting.........################')
        for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
            print('##### Fold',i+1)

            X_train, X_val = X.iloc[train_index], X.iloc[valid_index]
            y_train, y_val = y[train_index].values, y[valid_index].values

            if apply_scaler :
                X_train = scaler.transform(X_train)
                X_val = scaler.transform(X_val)
                
            if self.model_type == 'lgb':
                # Create a LightGBM dataset
                train_lgb_data = lgb.Dataset(X_train, label=y_train)
                val_lgb_data = lgb.Dataset(X_val, label=y_val)
                
                model = lgb.train(
                    params = lgb_params,
                    train_set = train_lgb_data,
                    num_boost_round = 50000,
                    valid_sets = [train_lgb_data, val_lgb_data],
                    verbose_eval = 10000,
                )
                preds = model.predict(X_val)
                preds_concat = np.concatenate(((1 - preds).reshape(-1, 1) , preds.reshape(-1, 1)), axis=1)
                oof[valid_index, :] = preds_concat
                
            elif self.model_type == 'xgb':
                model = XGBClassifier(random_state=42, tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor",**self.params )  
                model.fit(X_train, y_train,eval_set=[(X_val,y_val)],verbose=False)
                preds = model.predict_proba(X_val)
                oof[valid_index, :] = preds
                
            elif self.model_type == 'catboost':
                model = cb.CatBoostClassifier(**self.params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=1000)
                preds = model.predict_proba(X_val)
                oof[valid_index, :] = preds
                
            elif self.model_type == 'tabpfn':
                model = TabPFNClassifier(N_ensemble_configurations=64,device='cuda:0')
                model.fit(X_train, y_train)
                y_eval, preds = model.predict(X_val, return_winning_probability=True)
                preds_concat = np.concatenate(( preds.reshape(-1, 1) , (1 - preds).reshape(-1, 1) ), axis=1)
                oof[valid_index, :] = preds_concat
                
            else:
                raise ValueError("Invalid model_type. Supported types are 'lgb', 'xgb', 'catboost' and 'tabpfn' .")
                
            self.models.append(model)
            
        self.fitted = True
        
        print('---------------------------------------')
        print('ROC AUC Score:')
        print(roc_auc_score(y, oof[:,1]))
        print('Balance logloss:')
        print(balance_logloss(y, oof))
        print('---------------------------------------')
        
        return oof

    def predict_proba(self, test):
        if not self.fitted:
            raise ValueError("Model has not been fitted yet. Call fit() method first.")
            
        final_preds = []
        
        for i, model in enumerate(self.models):
            if self.model_type == 'lgb':            
                preds = model.predict(test)
                preds_concat = np.concatenate(((1 - preds).reshape(-1, 1) , preds.reshape(-1, 1)), axis=1)
                final_preds.append(preds_concat)
                
            elif self.model_type == 'xgb':
                preds = model.predict_proba(test)
                final_preds.append(preds)
                
            elif self.model_type == 'catboost':
                preds = model.predict_proba(test)
                final_preds.append(preds)
                
            elif self.model_type == 'tabpfn':
                y_eval, preds = model.predict(test, return_winning_probability=True)
                preds_concat = np.concatenate(( preds.reshape(-1, 1) , (1 - preds).reshape(-1, 1) ), axis=1)
                final_preds.append(preds_concat)
                
        return np.mean(final_preds, axis=0)

    def predict(self, test):
        return self.predict_proba(test)[:,1]

In [17]:
# LightGBM parameters
lgb_params = {
    'objective': 'binary', 
    'metric': 'binary_logloss', 
    'boosting': 'goss',
    'learning_rate': 0.09110460114828077,
    'num_leaves': 8,
    'feature_fraction': 0.4989639912997521,
    'bagging_fraction': 0.54872439795985,
    'lambda_l1': 1.4522184914523175, 
    'lambda_l2': 1.7873553090132748e-08,
    'n_jobs': -1,
    'early_stopping_rounds': 20,
    'is_unbalance':True, 
    'verbose': -1,
    'seed': 42,
}

# XGBoost parameters
xgb_params = {
    'n_estimators': 650,
    'learning_rate': 0.012208383405206188,
    'booster': 'gbtree',
    'lambda': 0.009968756668882757,
    'alpha': 0.02666266827121168,
    'subsample': 0.7097814108897231,
    'colsample_bytree': 0.7946945784285216,
    'max_depth': 3,
    'min_child_weight': 4,
    'eta': 0.5480204506554545,
    'gamma': 0.8788654128774149,
    'scale_pos_weight': 4.71,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'early_stopping_rounds': 150,
    'verbosity': 0,
    'scale_pos_weight': 509/108
}

# CatBoost parameters
catboost_params = {
    'iterations': 5000,
    'colsample_bylevel': 0.0513276895988184,
    'depth': 2,
    'learning_rate': 0.0256579773375401,
    'l2_leaf_reg': 8.22319805476255,
    'random_strength': 0.11327724457066,
    'od_type': "Iter", 
    'od_wait': 72,
    'bootstrap_type': "Bayesian",
    'grow_policy': 'SymmetricTree',
    'bagging_temperature': 9.58737431845122,
    'auto_class_weights': 'Balanced',
    'random_state': 42
}

# TabPFN parameters
tabpfn_params = {
}

# Create instances for each model type
kfold_model_lgb = KFoldModel(n_splits=10, model_type='lgb', params=lgb_params)
kfold_model_xgb = KFoldModel(n_splits=10, model_type='xgb', params=xgb_params)
kfold_model_catboost = KFoldModel(n_splits=5, model_type='catboost', params=catboost_params)
kfold_model_tabpfn = KFoldModel(n_splits=5, model_type='tabpfn', params=tabpfn_params)


In [18]:
# Assuming you have the training data X_train and labels y_train

# Fit LightGBM model
oof_lgb = kfold_model_lgb.fit(X, y)

# Fit XGBoost model
oof_xgb = kfold_model_xgb.fit(X, y)

# Fit CatBoost model
oof_catboost = kfold_model_catboost.fit(X, y)

# Fit TabPFN model
oof_tabpfn = kfold_model_tabpfn.fit(X, y)

# # Make predictions for test data using each model type
# predictions_lgb = kfold_model_lgb.predict_proba(test_data)
# predictions_xgb = kfold_model_xgb.predict_proba(test_data)
# predictions_catboost = kfold_model_catboost.predict_proba(test_data)
# predictions_tabpfn = kfold_model_tabpfn.predict_proba(test_data)

# # Average the predictions from all models (if you want to do ensemble predictions)
# final_predictions = (predictions_lgb + predictions_xgb + predictions_catboost + predictions_tabpfn) / 4

# # If you just want to use one model for prediction (e.g., LightGBM)
# final_predictions_lgb = kfold_model_lgb.predict(test_data)

################lgb train starting.........################
##### Fold 1
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[208]	training's binary_logloss: 0.0213267	valid_1's binary_logloss: 0.216985
##### Fold 2
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[289]	training's binary_logloss: 0.0164802	valid_1's binary_logloss: 0.0700064
##### Fold 3
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[112]	training's binary_logloss: 0.0544849	valid_1's binary_logloss: 0.172138
##### Fold 4
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[95]	training's binary_logloss: 0.0696639	valid_1's binary_logloss: 0.140002
##### Fold 5
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[215]	training's binary_logloss: 0.0221784	valid_1's binary_logloss: 0.114817
#####

In [19]:
# print(roc_auc_score(train['Class'] , kfold_model_catboost.predict(X)))
# print(balance_logloss(train['Class'] , kfold_model_catboost.predict_proba(X)))

In [20]:
# Make predictions for test data using each model type
predictions_lgb = kfold_model_lgb.predict_proba(test)
predictions_xgb = kfold_model_xgb.predict_proba(test)
predictions_catboost = kfold_model_catboost.predict_proba(test)
predictions_tabpfn = kfold_model_tabpfn.predict_proba(test)

# Average the predictions from all models (if you want to do ensemble predictions)
final_predictions = (predictions_lgb + predictions_xgb + predictions_catboost) / 3

In [21]:
# objective = partial(xgboost_optimizer, X=X, y=y, K=10)

# study = optuna.create_study(direction='minimize', sampler=sampler)
# study.optimize(objective, n_trials=400)

In [22]:
# import optuna
# import optuna.visualization as optuna_vis

# # visualize experiment logs
# def my_display_experiment_log(study):
#     optuna_vis.plot_optimization_history(study).show()
#     optuna_vis.plot_param_importances(study).show()
#     optuna_vis.plot_slice(study).show()
#     optuna_vis.plot_parallel_coordinate(study).show()
#     history = study.trials_dataframe()
# #     history.head()
#     print("Best Score: %.4f" % study.best_value)
#     print("Best params: ", study.best_trial.params)
# #     print(history[history.value == study.best_value])
#     history.to_csv('experiment_log.csv', index=False)

In [23]:
# my_display_experiment_log(study)

In [24]:
sample_submission = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv')
sample_submission[['class_0','class_1']] = final_predictions
sample_submission.to_csv('submission.csv',index=False)
sample_submission

,Id,class_0,class_1
0,00eed32682bb,0.536392,0.463608
1,010ebe33f668,0.536392,0.463608
2,02fa521e1838,0.536392,0.463608
3,040e15f562a2,0.536392,0.463608
4,046e85c7cc7f,0.536392,0.463608
